In [ ]:
!pip install category_encoders
# @TODO: make sure that the mapping is right for corresponding categorical dat

In [ ]:
import pandas as pd
import joblib
import json
import requests
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import drive
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor


drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp '/content/drive/MyDrive/FPL-Project/FPL-Utils.ipynb' /content/
%run /content/FPL-Utils.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
FPL_API_URL = "https://fantasy.premierleague.com/api/"
BOOTSTRAP_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [ ]:
# we want to only be concerned if the person played for at least 10 minutes
def convert_minutes(minutes):
    if minutes > 10:
        return 1
    return 0

In [ ]:
# GET THE NEXT WEEK DATA FROM THE API
fixtures = FPL_API_URL + "fixtures?event=2"
players = FPL_API_URL + "event/GW2/live"

# Get general player data
bootstrap_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
bootstrap_data = requests.get(bootstrap_url).json()

# Extract relevant player information
players = bootstrap_data['elements']
teams = bootstrap_data['teams']
teams = {team['id']: team['name'] for team in teams}

# Get all fixtures
fixtures_url = 'https://fantasy.premierleague.com/api/fixtures/'
fixtures_data = requests.get(fixtures_url).json()

# Determine the next gameweek number
next_gameweek = min([f['event'] for f in fixtures_data if not f['finished']])

# Filter fixtures for the next gameweek
upcoming_fixtures = [f for f in fixtures_data if f['event'] == next_gameweek]


In [ ]:
# will need to make call to get "_ex" data from previous season

# will need to make call to get the form data for this season
# call functions from
get_current_season_data("2024-25", "2023-24", "currentseason")
current_season_data = None

file_path = '/content/drive/MyDrive/FPL-Project/currentseason.csv'
if os.path.exists(file_path):
    print("File exists")
    current_season_data = pd.read_csv(file_path)
else:
    print("File does not exist")

Gameweek 2 not found
All Done, Saved the file!
File exists


In [ ]:
players[3]

In [ ]:
player_fixtures = []

for player in players:
  player_team_id = player['team']

  # Find the fixture involving the player's team
  for fixture in upcoming_fixtures:
    if fixture['team_h'] == player_team_id or fixture['team_a'] == player_team_id:
      player_fixtures.append({
        'name': player['first_name'] + " " + player['second_name'],
        'position': player['element_type'],  # Position, already encoded
        'team': teams[player_team_id],
        'xP': player['ep_this'],  # Assuming 'xP' corresponds to 'expected points'
        'value': player['now_cost'],  # 'value' is mapped to 'now_cost'
        'was_home': fixture['team_h'] == player_team_id,  # Check if the player was home
        'last_season_position': get_teams_previous_position("2023-24")(teams[player_team_id]),  # Previous season position
        'percent_value': 0 ,# @TODO need to calculate the number of points vs whole team
        'goals_scored_ex': player['goals_scored'],  # Goals scored in the previous season
        'assists_ex': player['assists'],  # Assists in the previous season
        'total_points_ex': 0,  # Total points in the previous season
        'minutes_ex': 0,  # Minutes played in the previous season
        'goals_conceded_ex': player['goals_conceded'],  # Goals conceded in the previous season
        'creativity_ex': 0,  # Creativity score in the previous season
        'influence_ex': 0,  # Influence score in the previous season
        'threat_ex': 0,  # Threat score in the previous season
        'bonus_ex': 0,  # Bonus points in the previous season
        'bps_ex': 0,  # Bonus Points System in the previous season
        'ict_index_ex': 0,  # ICT Index in the previous season
        'clean_sheets_ex': player['goals_conceded'],  # Clean sheets in the previous season
        'red_cards_ex': player['red_cards'],  # Red cards in the previous season
        'yellow_cards_ex': player['yellow_cards'],  # Yellow cards in the previous season
        'selected_by_percent_ex': player['selected_by_percent'],  # Selection percentage in the previous season WARNING
        'now_cost_ex': 0,  # Cost in the previous season
        'GW': next_gameweek,  # Gameweek number
        'opponent': teams[fixture['team_h']] if fixture['team_a'] == player_team_id else teams[fixture['team_a']],  # Opponent team
        'opponent_last_season_position': get_teams_previous_position("2023-24")(
            teams[fixture['team_h']] if fixture['team_a'] == player_team_id else teams[fixture['team_a']]
        ),
        'expected_assists': player['expected_assists_per_90'],  # Expected assists in the current season
        'expected_goal_involvements': player['expected_goal_involvements_per_90'],  # Expected goal involvements
        'expected_goals': player['expected_goals_per_90'],  # Expected goals in the current season
        'expected_goals_conceded': player['expected_goals_conceded_per_90'],  # Expected goals conceded
        'starts': player['starts_per_90'],  # Number of starts
        'selected_by_percent': player['selected_by_percent'],  # Selection percentage
        'total_points_5_match_form': 0,  # Total points in the last 5 matches
        'assists_5_match_form': 0,  # Assists in the last 5 matches
        'goals_5_match_form': 0,  # Goals in the last 5 matches
        'team_cumulative_goals_scored': 0,  # Team's cumulative goals scored
        'team_cumulative_goals_scored_5_match_form':0,  # Team's goals scored in the last 5 matches
        'team_cumulative_goals_conceded':0,  # Team's cumulative goals conceded
        'team_cumulative_goals_conceded_5_match_form':0,  # Team's goals conceded in the last 5 matches
        'gk_form_goals_conceded_5_match_form': 0,  # Goalkeeper's goals conceded in the last 5 matches
    })


In [ ]:
# read through previous season data and update the ex values in  players


In [ ]:
upcoming_gameweek_df = pd.DataFrame(player_fixtures)
# calculate the percent value of the season so far

# if current_season_data is not None:
#   # will need to find the form form, can only test this after GW1
#   upcoming_gameweek_df = create_goals_assissts_points_form_features(upcoming_gameweek_df)
#   gk_extract = create_goalkeepers_form_features(upcoming_gameweek_df)
#   conceded_df_combined = create_conceded_form_features(upcoming_gameweek_df)
#   scored_df_combined = create_scored_form_features(upcoming_gameweek_df)

#   upcoming_gameweek_df['team_cumulative_goals_scored'] = 0
#   upcoming_gameweek_df['team_cumulative_goals_scored_5_match_form'] = 0
#   upcoming_gameweek_df['team_cumulative_goals_conceded'] = 0
#   upcoming_gameweek_df['team_cumulative_goals_conceded_5_match_form'] = 0
#   upcoming_gameweek_df['gk_form_goals_conceded_5_match_form'] = 0

#   upcoming_gameweek_df.update(scored_df_combined[['season', 'team', 'GW', 'team_cumulative_goals_scored']])
#   upcoming_gameweek_df.update(conceded_df_combined[['season', 'team', 'GW', 'team_cumulative_goals_conceded']])
#   upcoming_gameweek_df.update(gk_extract[['season', 'name', 'GW', 'gk_form_goals_conceded_5_match_form']])

# upcoming_gameweek_df["last_season_position"] = upcoming_gameweek_df["team"].apply(get_teams_previous_position("2023-24"))
# upcoming_gameweek_df["opponent_last_season_position"] = upcoming_gameweek_df["opponent"].apply(
#       get_teams_previous_position("2023-24")
#   )

upcoming_gameweek_df.head()


,name,position,team,xP,value,was_home,last_season_position,percent_value,goals_scored_ex,assists_ex,total_points_ex,minutes_ex,goals_conceded_ex,creativity_ex,influence_ex,threat_ex,bonus_ex,bps_ex,ict_index_ex,clean_sheets_ex,red_cards_ex,yellow_cards_ex,selected_by_percent_ex,now_cost_ex,GW,opponent,opponent_last_season_position,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,selected_by_percent,total_points_5_match_form,assists_5_match_form,goals_5_match_form,team_cumulative_goals_scored,team_cumulative_goals_scored_5_match_form,team_cumulative_goals_conceded,team_cumulative_goals_conceded_5_match_form,gk_form_goals_conceded_5_match_form
0,Fábio Ferreira Vieira,3,Arsenal,2.3,55,False,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,2,Aston Villa,4,0.00,0.00,0.00,0.00,0.0,0.0,0,0,0,0,0,0,0,0
1,Gabriel Fernando de Jesus,4,Arsenal,3.2,69,False,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.9,0,2,Aston Villa,4,0.00,0.00,0.00,2.70,0.0,1.9,0,0,0,0,0,0,0,0
2,Gabriel dos Santos Magalhães,2,Arsenal,4.2,60,False,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.2,0,2,Aston Villa,4,0.02,0.14,0.12,0.47,1.0,13.2,0,0,0,0,0,0,0,0
3,Kai Havertz,4,Arsenal,3.5,81,False,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,16.4,0,2,Aston Villa,4,0.04,0.49,0.45,0.47,1.0,16.4,0,0,0,0,0,0,0,0
4,Karl Hein,1,Arsenal,0.0,40,False,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,2,Aston Villa,4,0.00,0.00,0.00,0.00,0.0,0.1,0,0,0,0,0,0,0,0


In [ ]:
X = data.drop('total_points', axis=1)
y = data['total_points']

# ensure that the upcoming gameweek df has the same columns
upcoming_gameweek_df = upcoming_gameweek_df[X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
factorized_mappings = {}

# Convert categorical columns to numerical
for col in X_train.columns:
    if X_train[col].dtype == "object":
        # X_train[col], factorized_mappings[col] = pd.factorize(X_train[col])
        # # X_test[col], _ = pd.factorize(X_test[col])
        # X_test[col] = factorized_mappings[col].get_indexer(X_test[col])

        # Factorize the training data
        X_train[col], unique_values = pd.factorize(X_train[col])
        factorized_mappings[col] = unique_values

        # Factorize the test data with the same categories
        X_test[col] = pd.Categorical(X_test[col], categories=unique_values).codes

        # Factorize the upcoming gameweek data with the same categories
        upcoming_gameweek_df[col] = pd.Categorical(upcoming_gameweek_df[col], categories=unique_values).codes

# for col in upcoming_gameweek_df.columns:
#     if col in factorized_mappings:
#         upcoming_gameweek_df[col] = factorized_mappings[col].get_indexer(upcoming_gameweek_df[col])

reverse_factorized_mappings = {col: dict(enumerate(factorized_mappings[col])) for col in factorized_mappings}

upcoming_gameweek_df.head()

KeyError: "['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat', 'transfers_balance', 'transfers_in', 'transfers_out', 'yellow_cards', 'percent_value', 'match_status', 'season', 'element_type_ex'] not in index"

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/FPL-Project/updated_cleaned_previous_seasons_gameweeks_with_features.csv')

# we need to fill the NaN values
# Columns that aren't needed for training
unrequired = ["assists", "bonus", "bps",
              "clean_sheets", "goals_conceded", "goals_scored", "minutes",
              "own_goals", "penalties_missed", "penalties_saved", "red_cards",
              "saves", "yellow_cards", "team_a_score", "team_h_score",
              "transfers_balance", "transfers_in", "transfers_out", "round",
              "match_status", "creativity", "ict_index", "influence", "threat",
              "fixture", "selected", "season", "opponent_team",
              'element', 'element_type_ex', 'percent_value',
              ]

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

# Ensure consistent data types
data['was_home'] = data['was_home'].replace({True: 1, False: 0})
data['position'] = data['position'].replace({'GK': 1, 'DEF': 2, 'MID': 3, 'FWD': 4, 'GKP': 1})

# Drop unnecessary columns
data.drop(unrequired, axis=1, inplace=True)
data.drop(["kickoff_time"], axis=1, inplace=True)
data.fillna(0, inplace=True)

data.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,last_season_position,percent_value,match_status,goals_scored_ex,assists_ex,total_points_ex,minutes_ex,goals_conceded_ex,creativity_ex,influence_ex,threat_ex,bonus_ex,bps_ex,ict_index_ex,clean_sheets_ex,red_cards_ex,yellow_cards_ex,selected_by_percent_ex,now_cost_ex,season,GW,opponent,opponent_last_season_position,element_type_ex,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,total_points_5_match_form,assists_5_match_form,goals_5_match_form,team_cumulative_goals_scored,team_cumulative_goals_scored_5_match_form,team_cumulative_goals_conceded,team_cumulative_goals_conceded_5_match_form,gk_form_goals_conceded_5_match_form
0,Aaron Connolly,4,Brighton,0.5,0,0,-3,0,0.3,78,7,2,0,3.4,1.2,45,5,0,0,0,0,1,0,32205,3,1,32.0,1,0,0,0,55,1,0,15,3.703704,0.0,3.0,3.0,61.0,1250.0,22.0,91.4,150.6,530.0,5.0,146.0,76.0,5.0,0.0,0.0,2.7,42.0,2020-21,1,Chelsea,4,0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,1,1.0,3,3.0,0.0
1,Aaron Connolly,4,Brighton,4.0,0,2,27,1,11.3,78,16,0,1,6.9,34.8,89,14,0,0,0,0,2,0,35030,3,0,23.0,8,-1161,5332,6493,55,0,0,15,11.764706,3.0,3.0,3.0,61.0,1250.0,22.0,91.4,150.6,530.0,5.0,146.0,76.0,5.0,0.0,0.0,2.7,42.0,2020-21,2,Newcastle,13,0,0.0,0.0,0.0,0.0,0.0,4.500000,0.0,0.500000,4,4.0,3,3.0,0.0
2,Aaron Connolly,4,Brighton,2.7,0,0,2,0,12.1,78,19,2,0,1.9,0.0,73,13,0,0,0,0,3,0,55356,3,2,8.0,2,13526,26823,13297,55,1,0,15,6.451613,0.0,3.0,3.0,61.0,1250.0,22.0,91.4,150.6,530.0,5.0,146.0,76.0,5.0,0.0,0.0,2.7,42.0,2020-21,3,Man Utd,21,0,0.0,0.0,0.0,0.0,0.0,3.666667,0.0,0.333333,6,6.0,6,6.0,0.0
3,Aaron Connolly,4,Brighton,2.7,0,0,7,0,0.3,78,32,3,0,0.7,3.0,65,7,0,0,0,0,4,0,55125,2,4,4.0,2,-1311,10399,11710,55,0,0,15,6.666667,3.0,3.0,3.0,61.0,1250.0,22.0,91.4,150.6,530.0,5.0,146.0,76.0,5.0,0.0,0.0,2.7,42.0,2020-21,4,Everton,12,0,0.0,0.0,0.0,0.0,0.0,3.250000,0.0,0.250000,8,8.0,10,10.0,0.0
4,Aaron Connolly,4,Brighton,3.0,1,0,13,0,10.3,78,40,0,0,3.0,17.2,12,6,0,0,0,0,5,0,46774,1,1,2.0,4,-8992,5860,14852,55,0,0,15,12.500000,1.0,3.0,3.0,61.0,1250.0,22.0,91.4,150.6,530.0,5.0,146.0,76.0,5.0,0.0,0.0,2.7,42.0,2020-21,5,Crystal Palace,14,0,0.0,0.0,0.0,0.0,0.0,3.400000,0.2,0.200000,9,9.0,11,11.0,0.0


In [ ]:
# TRYING SOMETHING NEW
# FILTER OUT BY POSITION

# try to print the total number of people in each position
print(data['position'].value_counts())

# filter out by position FWD
data = data[data['position'] == 4]
print(data['position'].value_counts())

print(upcoming_gameweek_df['position'].value_counts())

upcoming_gameweek_df  = upcoming_gameweek_df[upcoming_gameweek_df['position'] == 4]
print(upcoming_gameweek_df['position'].value_counts())



position
3    31795
2    26581
4     9624
1     8469
Name: count, dtype: int64
position
4    9624
Name: count, dtype: int64
position
4    72
Name: count, dtype: int64
position
4    72
Name: count, dtype: int64


In [ ]:
# THIS IS FOR FINDING THE BEST FEATURES PER POSITION
X = data.drop('total_points', axis=1)
y = data['total_points']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# feature importance
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Convert categorical columns to numerical
for col in X_train.columns:
    if X_train[col].dtype == "object":
        # X_train[col], factorized_mappings[col] = pd.factorize(X_train[col])
        # # X_test[col], _ = pd.factorize(X_test[col])
        # X_test[col] = factorized_mappings[col].get_indexer(X_test[col])

        # Factorize the training data
        X_train[col], unique_values = pd.factorize(X_train[col])
        factorized_mappings[col] = unique_values

        # Factorize the test data with the same categories
        X_test[col] = pd.Categorical(X_test[col], categories=unique_values).codes

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
rf_model.fit(X_train, y_train)

importances = rf_model.feature_importances_
feature_importance = pd.Series(importances, index=X.columns).sort_values(ascending=False)
print(feature_importance)

bps                                            0.842151
percent_value                                  0.066441
assists                                        0.031312
bonus                                          0.023741
minutes                                        0.023058
influence                                      0.003972
yellow_cards                                   0.003857
creativity                                     0.001002
goals_scored                                   0.000741
assists_5_match_form                           0.000259
total_points_5_match_form                      0.000229
penalties_missed                               0.000172
team_a_score                                   0.000172
red_cards                                      0.000160
ict_index                                      0.000147
xP                                             0.000128
threat                                         0.000105
name                                           0

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# # Define the features (X) and the target (y)
# X = data.drop('total_points', axis=1)
# y = data['total_points']

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Save the fitted scaler
joblib.dump(scaler, 'scaler.pkl')

# Define the model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 6.6883 - val_loss: 3.5166
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.2764 - val_loss: 3.2543
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2491 - val_loss: 3.2878
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8549 - val_loss: 3.1951
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5426 - val_loss: 3.2178
Epoch 6/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.8763 - val_loss: 3.1680
Epoch 7/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5581 - val_loss: 3.1947
Epoch 8/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4236 - val_loss: 3.2461
Epoch 9/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3005 - val_loss: 3.1838
Epoch 10/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5675 - val_loss: 3.2453
Epoch 11/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6417 - val_loss: 3.2336
Epoch 12/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4115 - val_lo

In [ ]:
upcoming_gameweek_df.columns

In [ ]:
# apply the same scaling

upcoming_gameweek_df_scaled = scaler.transform(upcoming_gameweek_df)

# make predictions
predictions = model.predict(upcoming_gameweek_df_scaled)
upcoming_gameweek_df['predicted_total_points'] = predictions
upcoming_gameweek_df.head()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


,name,position,team,xP,value,was_home,last_season_position,goals_scored_ex,assists_ex,total_points_ex,minutes_ex,goals_conceded_ex,creativity_ex,influence_ex,threat_ex,bonus_ex,bps_ex,ict_index_ex,clean_sheets_ex,red_cards_ex,yellow_cards_ex,selected_by_percent_ex,now_cost_ex,GW,opponent,opponent_last_season_position,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,total_points_5_match_form,assists_5_match_form,goals_5_match_form,team_cumulative_goals_scored,team_cumulative_goals_scored_5_match_form,team_cumulative_goals_conceded,team_cumulative_goals_conceded_5_match_form,gk_form_goals_conceded_5_match_form,predicted_total_points
1,52,4,8,3.2,69,False,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.9,0,2,21,4,0.00,0.00,0.00,2.70,0.00,0,0,0,0,0,0,0,0,1.162345
3,163,4,8,3.5,81,False,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,16.4,0,2,21,4,0.04,0.49,0.45,0.47,1.00,0,0,0,0,0,0,0,0,1.283173
10,105,4,8,3.0,60,False,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.3,0,2,21,4,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0.894076
36,176,4,2,2.0,60,True,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6,0,2,1,2,0.00,1.22,1.22,3.76,0.00,0,0,0,0,0,0,0,0,2.282011
56,22,4,2,2.7,90,True,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,32.3,0,2,1,2,0.00,0.00,0.00,1.89,1.48,0,0,0,0,0,0,0,0,1.338846


In [ ]:
top_5_players_by_position = upcoming_gameweek_df.groupby('position').apply(
    lambda df: df.nlargest(10, 'predicted_total_points')
)

# Reset index to make it cleaner
top_5_players_by_position = top_5_players_by_position.reset_index(drop=True)

# Reverse mapping to get the original position names using reverse_factorized_mappings
top_5_players_by_position['name'] = top_5_players_by_position['name'].map(reverse_factorized_mappings['name'])

# Similarly, if you have other columns that were factorized, you can reverse-map them as well:
# Example for a 'team' column (if it was factorized):
top_5_players_by_position['position'] = top_5_players_by_position['position'].replace({1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'})

top_5_players_by_position['team']  = top_5_players_by_position['team'].map(reverse_factorized_mappings['team'])

# Display the top 5 players in each position
top_5_players_by_position[['name', 'position', 'team', 'value', 'predicted_total_points']]

,name,position,team,value,predicted_total_points
0,Richarlison de Andrade,FWD,Spurs,70,33.917801
1,Jay Stansfield,FWD,Fulham,50,12.458311
2,Danny Ings,FWD,West Ham,50,9.114031
3,Taiwo Awoniyi,FWD,Nott'm Forest,60,2.531907
4,NaN,FWD,Brentford,60,2.489622
5,Cameron Archer,FWD,Southampton,50,2.295982
6,Jhon Durán,FWD,Aston Villa,60,2.282011
7,Chris Wood,FWD,Nott'm Forest,60,2.219743
8,Raúl Jiménez,FWD,Fulham,55,1.984394
9,NaN,FWD,Spurs,75,1.863674


### Random Forest

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [ ]:
y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")

MAE: 0.8746441558441559
MSE: 3.210950181818182


In [ ]:
upcoming_gameweek_df = upcoming_gameweek_df[X.columns]  # Ensure columns match
upcoming_predictions = rf_model.predict(upcoming_gameweek_df)

# Add predictions to the DataFrame
upcoming_gameweek_df['predicted_total_points'] = upcoming_predictions
print(upcoming_gameweek_df.head())

top_5_players_by_position = upcoming_gameweek_df.groupby('position').apply(
    lambda df: df.nlargest(5, 'predicted_total_points')
)

# Reset index to make it cleaner
top_5_players_by_position = top_5_players_by_position.reset_index(drop=True)

# Reverse mapping to get the original position names using reverse_factorized_mappings
top_5_players_by_position['name'] = top_5_players_by_position['name'].map(reverse_factorized_mappings['name'])

# Similarly, if you have other columns that were factorized, you can reverse-map them as well:
# Example for a 'team' column (if it was factorized):
top_5_players_by_position['position'] = top_5_players_by_position['position'].replace({1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'})

top_5_players_by_position['team']  = top_5_players_by_position['team'].map(reverse_factorized_mappings['team'])

# Display the top 5 players in each position
top_5_players_by_position[['name', 'position', 'team', 'value', 'predicted_total_points']]

    name  position  team   xP  value  was_home  last_season_position  \
1     52         4     8  3.2     69     False                     2   
3    163         4     8  3.5     81     False                     2   
10   105         4     8  3.0     60     False                     2   
36   176         4     2  2.0     60      True                     4   
56    22         4     2  2.7     90      True                     4   

    goals_scored_ex  assists_ex  total_points_ex  minutes_ex  \
1                 0           0                0           0   
3                 1           1                0           0   
10                0           0                0           0   
36                1           0                0           0   
56                0           0                0           0   

    goals_conceded_ex  creativity_ex  influence_ex  threat_ex  bonus_ex  \
1                   0              0             0          0         0   
3                   0           

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


,name,position,team,value,predicted_total_points
0,Richarlison de Andrade,FWD,Spurs,70,5.67
1,Danny Ings,FWD,West Ham,50,4.36
2,Erling Haaland,FWD,Man City,150,4.34
3,Liam Delap,FWD,NaN,55,4.25
4,Alexander Isak,FWD,Newcastle,85,4.14
